This notebook is not used operationally or for any validation, its only purpose is to have a clear understanding of the core functions of the AA workflow. The outputs and dimensions of each main step can thus be identified here.

In [1]:
import datetime
import pandas as pd
import geopandas as gpd

from config.params import Params

from helper_fns import (
    read_forecasts_locally,
    read_observations_locally,
    aggregate_by_district,
    merge_un_biased_probs,
    merge_probabilities_triggers_dashboard,
)

# from analysis.aoi import AnalysisArea, AnalysisAreaData, Country
from hip.analysis.analyses.drought import (
    get_accumulation_periods,
    run_accumulation_index,
    run_gamma_standardization,
    run_bias_correction,
    compute_probabilities,
)

%cd ../

c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\aa-env\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.3
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


c:\Users\amine.barkaoui\OneDrive - World Food Programme\Documents\GitHub\anticipatory-action


In [2]:
params = Params(iso='MOZ', issue=10, index='SPI')

In [3]:
# To replace with HDC dataset
forecasts = read_forecasts_locally(
    f"data/{params.iso}/forecast/{params.iso}_SAB_tp_ecmwf_{str(params.issue).zfill(2)}/*.nc"
)
forecasts

<xarray.Dataset>
Dimensions:    (longitude: 56, latitude: 80, ensemble: 51, time: 6580)
Coordinates:
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
  * ensemble   (ensemble) float64 0.0 1.0 2.0 3.0 4.0 ... 47.0 48.0 49.0 50.0
  * time       (time) datetime64[ns] 1993-10-01T12:00:00 ... 2024-04-30T12:00:00
Data variables:
    tp         (time, ensemble, latitude, longitude) float32 27.3 ... 8.367
Attributes:
    CDI:          Climate Data Interface version 1.9.8 (https://mpimet.mpg.de...
    Conventions:  CF-1.6
    history:      Mon Oct 19 19:56:58 2020: cdo -O -L -r -selmon,10,11,12,1,2...
    NCO:          netCDF Operators version 4.9.2 (Homepage = http://nco.sf.ne...
    CDO:          Climate Data Operators version 1.9.8 (https://mpimet.mpg.de...

In [4]:
# To replace with CHIRPS (rfh_daily for DRYSPELL or r1h_dekad if SPI)
observations = read_observations_locally(f"data/{params.iso}/chirps")
observations

<xarray.Dataset>
Dimensions:    (time: 15491, longitude: 56, latitude: 80)
Coordinates:
  * time       (time) datetime64[ns] 1981-01-01 1981-01-02 ... 2023-05-31
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
Data variables:
    precip     (time, latitude, longitude) float32 0.0 0.0 0.0 ... nan nan nan
Attributes: (12/17)
    CDI:               Climate Data Interface version 1.9.10 (https://mpimet....
    Conventions:       CF-1.6
    institution:       Climate Hazards Group.  University of California at Sa...
    history:           Wed Jul 14 16:25:15 2021: cdo -O -r -L mergetime Remap...
    title:             CHIRPS Version 2.0
    version:           Version 2.0
    ...                ...
    comments:           time variable denotes the first day of the given day.
    acknowledgements:  The Climate Hazards Group InfraRed Precipitation with ...
    ftp_url:           ftp://chg-ftpout.geog.ucsb.edu/pub/org/chg/products/CH...
    website:           http://chg.geog.ucsb.edu/data/chirps/index.html
    faq:               http://chg-wiki.geog.ucsb.edu/wiki/CHIRPS_FAQ
    CDO:               Climate Data Operators version 1.9.10 (https://mpimet....

In [5]:
# Read triggers file
triggers_df = pd.read_csv(
    f"data/{params.iso}/outputs/Plots/triggers.aa.python.spi.dryspell.2022.csv"
)

In [6]:
gdf = gpd.read_file(
    f"data/{params.iso}/shapefiles/moz_admbnda_2019_SHP/moz_admbnda_adm2_2019.shp"
)

In [35]:
# Get accumulation periods (DJ, JF, FM, DJF, JFM...)
accumulation_periods = get_accumulation_periods(
    forecasts,
    params.start_season,
    params.end_season,
    params.min_index_period,
    params.max_index_period,
)

In [38]:
# Get single use case
period_name, period_months = list(accumulation_periods.items())[6]
period_name, period_months

('JFM', (1, 2, 3))

In [9]:
# Remove 1980 season to harmonize observations between different indexes
if int(params.issue) >= params.end_season:
    observations = observations.where(
        observations.time.dt.date >= datetime.date(1981, 10, 1), drop=True
    )

In [10]:
# Accumulation
accumulation_fc = run_accumulation_index(
    forecasts, params.aggregate, period_months, forecasts=True
)
accumulation_obs = run_accumulation_index(
    observations, params.aggregate, period_months
)
display(accumulation_fc)

<xarray.Dataset>
Dimensions:    (time: 31, ensemble: 51, latitude: 80, longitude: 56)
Coordinates:
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
  * ensemble   (ensemble) float64 0.0 1.0 2.0 3.0 4.0 ... 47.0 48.0 49.0 50.0
  * time       (time) datetime64[ns] 1993-01-01 1994-01-01 ... 2023-01-01
Data variables:
    tp         (time, ensemble, latitude, longitude) float32 500.0 ... 1.005e+03

In [11]:
# Remove inconsistent observations
accumulation_obs = accumulation_obs.sel(
    time=slice(datetime.date(1979, 1, 1), datetime.date(params.year - 1, 12, 31))
)

In [12]:
# Anomaly
anomaly_fc = run_gamma_standardization(
    accumulation_fc, params.calibration_start, params.calibration_stop, members=True
)
anomaly_obs = run_gamma_standardization(
    accumulation_obs,
    params.calibration_start,
    params.calibration_stop,
)
display(anomaly_fc)

<xarray.Dataset>
Dimensions:    (latitude: 80, longitude: 56, time: 957)
Coordinates:
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
    ensemble   (time) float64 0.0 1.0 2.0 3.0 4.0 ... 46.0 47.0 48.0 49.0 50.0
  * time       (time) datetime64[ns] 1993-01-01 1993-01-02 ... 2023-02-20
Data variables:
    tp         (latitude, longitude, time) float64 114.0 276.0 ... 2.659e+03

In [13]:
# Probabilities without Bias Correction
probabilities = compute_probabilities(
    anomaly_fc.where(anomaly_fc.time.dt.year == params.year, drop=True),
    levels=params.intensity_thresholds,
).round(2)
display(probabilities)

<xarray.Dataset>
Dimensions:    (longitude: 56, latitude: 80, category: 3)
Coordinates:
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
  * category   (category) object 'Severo' 'Moderado' 'Leve'
Data variables:
    tp         (category, latitude, longitude) float64 0.12 0.12 ... 0.31 0.33

In [16]:
# Bias correction
index_bc = run_bias_correction(
    anomaly_fc, 
    anomaly_obs, 
    params.end_season,
    params.year,
    int(params.issue),
    enso=True,
)
display(index_bc)

c:\Users\amine.barkaoui\AppData\Local\miniconda3\envs\aa-env\lib\site-packages\xclim\sdba\utils.py:317: RuntimeWarning: All-NaN slice encountered in interp_on_quantiles
  warn(


<xarray.Dataset>
Dimensions:    (latitude: 80, longitude: 56, time: 51)
Coordinates:
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * time       (time) datetime64[ns] 2022-01-01 2022-01-02 ... 2022-02-20
    ensemble   (time) float64 0.0 1.0 2.0 3.0 4.0 ... 46.0 47.0 48.0 49.0 50.0
Data variables:
    scen       (latitude, time, longitude) float64 0.2559 311.4 ... nan nan

In [17]:
# Probabilities after Bias Correction
probabilities_bc = compute_probabilities(
    index_bc, levels=params.intensity_thresholds
).round(2)
display(probabilities_bc)

<xarray.Dataset>
Dimensions:    (latitude: 80, longitude: 56, category: 3)
Coordinates:
  * latitude   (latitude) float32 -29.88 -29.62 -29.38 ... -10.62 -10.38 -10.12
  * longitude  (longitude) float32 28.12 28.38 28.62 28.88 ... 41.38 41.62 41.88
  * category   (category) object 'Severo' 'Moderado' 'Leve'
Data variables:
    scen       (category, latitude, longitude) float64 0.02 0.02 0.1 ... nan nan

In [17]:
# Aggregate by district
probs_district = aggregate_by_district(probabilities, gdf, params)
probs_bc_district = aggregate_by_district(probabilities_bc, gdf, params)

# Build single xarray with merged unbiased/biased probabilities
probs_by_district = merge_un_biased_probs(
    probs_district, probs_bc_district, params, period_name
)
display(probs_by_district)

<xarray.Dataset>
Dimensions:      (district: 11, category: 3, issue: 1, index: 1)
Coordinates:
  * district     (district) object 'Caia' 'Changara' ... 'Marara' 'Massingir'
  * category     (category) object 'Leve' 'Moderado' 'Severo'
  * issue        (issue) int64 10
  * index        (index) object 'SPI JFM'
    spatial_ref  int32 0
Data variables:
    prob         (index, district, category, issue) float64 0.08 0.0 ... 0.06

In [19]:
# Merge probabilities with triggers
probs_df, merged_df = merge_probabilities_triggers_dashboard(
    probs_by_district, triggers_df, params, period_name
)

In [23]:
probs_df

,district,category,issue,index,prob,year,aggregation
0,Caia,Leve,10,SPI JFM,0.08,2022,SPI 3
1,Caia,Moderado,10,SPI JFM,0.00,2022,SPI 3
2,Caia,Severo,10,SPI JFM,0.00,2022,SPI 3
3,Changara,Leve,10,SPI JFM,0.00,2022,SPI 3
4,Changara,Moderado,10,SPI JFM,0.00,2022,SPI 3
5,Changara,Severo,10,SPI JFM,0.00,2022,SPI 3
6,Chemba,Leve,10,SPI JFM,0.09,2022,SPI 3
7,Chemba,Moderado,10,SPI JFM,0.01,2022,SPI 3
8,Chemba,Severo,10,SPI JFM,0.00,2022,SPI 3
9,Chibuto,Leve,10,SPI JFM,0.11,2022,SPI 3


In [24]:
merged_df

,district,index,category,issue,trigger,trigger_value,lead_time,HR,type,Window,FR,prob,year,trigger_type
0,Chibuto,SPI JFM,Leve,10,trigger2,0.23,1,-0.800000,SPI,Janela 2,0.363636,0.11,2022-23,Acionadores de Crise
1,Chibuto,SPI JFM,Moderado,10,trigger1,0.25,1,-0.750000,SPI,Janela 2,0.375000,0.08,2022-23,Acionadores de Crise
2,Mabalane,SPI JFM,Leve,10,trigger1,0.00,1,-0.714286,SPI,Janela 2,0.300000,0.13,2022-23,Acionadores Gerais
3,Mapai,SPI JFM,Leve,10,trigger1,0.19,1,-0.833333,SPI,Janela 2,0.272727,0.14,2022-23,Acionadores Gerais
